In [24]:
import os
from shutil import copy2
import csv

In [25]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # The %tensorflow_version magic only works in colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt


In [39]:
import tensorflow_hub as hub

In [27]:

tf.__version__


'2.9.1'

In [28]:
data_root = ('dataset4')

In [29]:
IMAGE_SHAPE = (224, 224) # (height, width) in no. of pixels

In [30]:
TRAINING_DATA_DIR = str(data_root)

In [31]:
datagen_kwargs = dict(rescale=1./255, validation_split=.60)

In [32]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset='validation',
batch_size=16,
shuffle=True,
seed=123,
class_mode='binary', # categorical or binary
target_size=IMAGE_SHAPE
)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs,
                                                                rotation_range=15, 
                                                                zoom_range=0.1,
	                                                              width_shift_range=0.15, 
                                                                height_shift_range=0.15, 
                                                                shear_range=0.1,
                                                                fill_mode="nearest",
                                                                horizontal_flip=True
                                                                )
train_generator = train_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset='training',
batch_size=16,
shuffle=True,
seed=123,
class_mode='binary', # categorical or binary
target_size=IMAGE_SHAPE)

Found 5112 images belonging to 2 classes.
Found 3409 images belonging to 2 classes.


In [33]:
from PIL import Image

for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((16, 224, 224, 3), (16,))

In [34]:
print (train_generator.class_indices)

{'Real': 0, 'Spoof': 1}


In [35]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

In [36]:
f = open('labels.txt')
print(f.read()) 

Real
Spoof


In [37]:
IMAGE_SIZE = 224

In [40]:
base_model = hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_075_224/classification/5", 
                 trainable=True)

In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math


In [23]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args

In [20]:
dim_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform',
                         name='learning_rate')

dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')

dim_num_dense_nodes = Integer(low=5, high=512, name='num_dense_nodes')

dim_activation = Categorical(categories=['relu', 'sigmoid'],
                             name='activation')

In [21]:
dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_dense_nodes,
              dim_activation]

In [22]:
default_parameters = [1e-5, 1, 16, 'relu']

In [23]:
#path_best_model = '/content/drive/MyDrive/Liveness/Model/best_model.h5'
#local
path_best_model = 'best_model.h5'

In [24]:
best_accuracy = 0.0

In [25]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

In [45]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

In [27]:
def log_dir_name(learning_rate, num_dense_layers,
                 num_dense_nodes, activation):

    # The dir-name for the TensorBoard log-dir.
    s = "./19_logs/lr_{0:.0e}_layers_{1}_nodes_{2}_{3}/"

    # Insert all the hyper-parameters in the dir-name.
    log_dir = s.format(learning_rate,
                       num_dense_layers,
                       num_dense_nodes,
                       activation)

    return log_dir

In [28]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Input
from tensorflow.keras.layers import Reshape, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [29]:
def create_model(learning_rate, num_dense_layers,
                 num_dense_nodes, activation):
    """
    Hyper-parameters:
    learning_rate:     Learning-rate for the optimizer.
    num_dense_layers:  Number of dense layers.
    num_dense_nodes:   Number of nodes in each dense layer.
    activation:        Activation function for all layers.
    """
    
    # Start construction of a Keras Sequential model.
    model = Sequential()

    # Add an input layer which is similar to a feed_dict in TensorFlow.
    # Note that the input-shape must be a tuple containing the image-size.
    model.add(base_model)

    # Add fully-connected / dense layers.
    # The number of layers is a hyper-parameter we want to optimize.
    for i in range(num_dense_layers):
        # Name of the layer. This is not really necessary
        # because Keras should give them unique names.
        name = 'layer_dense_{0}'.format(i+1)

        # Add the dense / fully-connected layer to the model.
        # This has two hyper-parameters we want to optimize:
        # The number of nodes and the activation function.
        model.add(Dense(num_dense_nodes,
                        activation=activation,
                        name=name))

    # Last fully-connected / dense layer with softmax-activation
    # for use in classification.
    
    model.add(Dense(train_generator.num_classes-1, activation='sigmoid'))
    
    # Use the Adam method for training the network.
    # We want to find the best learning-rate for the Adam method.
    optimizer = Adam(learning_rate=learning_rate)
    
    # In Keras we need to compile the model so it can be trained.
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy', precision, recall])
    
    return model

In [30]:
@use_named_args(dimensions=dimensions)

def fitness(learning_rate, num_dense_layers,
            num_dense_nodes, activation):
    """
    Hyper-parameters:
    learning_rate:     Learning-rate for the optimizer.
    num_dense_layers:  Number of dense layers.
    num_dense_nodes:   Number of nodes in each dense layer.
    activation:        Activation function for all layers.
    """

    # Print the hyper-parameters.
    print('learning rate: {0:.1e}'.format(learning_rate))
    print('num_dense_layers:', num_dense_layers)
    print('num_dense_nodes:', num_dense_nodes)
    print('activation:', activation)
    print()
    
    # Create the neural network with these hyper-parameters.
    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_dense_nodes=num_dense_nodes,
                         activation=activation)

    # Dir-name for the TensorBoard log-files.
    log_dir = log_dir_name(learning_rate, num_dense_layers,
                           num_dense_nodes, activation)
    
    # Create a callback-function for Keras which will be
    # run after each epoch has ended during training.
    # This saves the log-files for TensorBoard.
    # Note that there are complications when histogram_freq=1.
    # It might give strange errors and it also does not properly
    # support Keras data-generators for the validation-set.
    callback_log = TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True,
        write_grads=False,
        write_images=False)
   
    # Use Keras to train the model.
    history = model.fit(train_generator,
                        epochs=19,
                        validation_data=valid_generator,
                        steps_per_epoch=steps_per_epoch,
                        validation_steps=val_steps_per_epoch,
                        callbacks=[callback_log])
    


    # Get the classification accuracy on the validation-set
    # after the last training-epoch.
    accuracy = history.history['val_accuracy'][-1]

    # Print the classification accuracy.
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()

    # Save the model if it improves on the best-found performance.
    # We use the global keyword so we update the variable outside
    # of this function.
    global best_accuracy

    # If the classification accuracy of the saved model is improved ...
    if accuracy > best_accuracy:
        # Save the new model to harddisk.
        model.save(path_best_model)
        
        # Update the classification accuracy.
        best_accuracy = accuracy

    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    
    # NOTE: Scikit-optimize does minimization so it tries to
    # find a set of hyper-parameters with the LOWEST fitness-value.
    # Because we are interested in the HIGHEST classification
    # accuracy, we need to negate this number so it can be minimized.
    return -accuracy

In [31]:
fitness(x=default_parameters)

learning rate: 1.0e-05
num_dense_layers: 1
num_dense_nodes: 16
activation: relu

Epoch 1/19
122/122 [==============================] - 35s 225ms/step - loss: 0.7412 - accuracy: 0.6677 - precision: 0.6670 - recall: 0.6684 - val_loss: 0.6200 - val_accuracy: 0.7479 - val_precision: 0.8371 - val_recall: 0.6157
Epoch 2/19
122/122 [==============================] - 19s 157ms/step - loss: 0.5360 - accuracy: 0.8065 - precision: 0.8105 - recall: 0.7996 - val_loss: 0.4203 - val_accuracy: 0.8595 - val_precision: 0.8425 - val_recall: 0.8843
Epoch 3/19
122/122 [==============================] - 18s 149ms/step - loss: 0.4315 - accuracy: 0.8658 - precision: 0.8642 - recall: 0.8678 - val_loss: 0.3396 - val_accuracy: 0.9091 - val_precision: 0.8667 - val_recall: 0.9669
Epoch 4/19
122/122 [==============================] - 19s 151ms/step - loss: 0.3748 - accuracy: 0.8906 - precision: 0.8962 - recall: 0.8833 - val_loss: 0.3286 - val_accuracy: 0.9008 - val_precision: 0.8440 - val_recall: 0.9835
Epoch 5/19


-0.9896694421768188

In [ ]:
%%time
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=50,
                            x0=default_parameters)

learning rate: 1.0e-05
num_dense_layers: 1
num_dense_nodes: 16
activation: relu

Epoch 1/19
122/122 [==============================] - 23s 158ms/step - loss: 0.5200 - accuracy: 0.8050 - precision: 0.8376 - recall: 0.8479 - val_loss: 0.2858 - val_accuracy: 0.9483 - val_precision: 0.9157 - val_recall: 0.9876
Epoch 2/19
122/122 [==============================] - 19s 152ms/step - loss: 0.2870 - accuracy: 0.9345 - precision: 0.9304 - recall: 0.9390 - val_loss: 0.2182 - val_accuracy: 0.9669 - val_precision: 0.9414 - val_recall: 0.9959
Epoch 3/19
122/122 [==============================] - 18s 150ms/step - loss: 0.2305 - accuracy: 0.9541 - precision: 0.9536 - recall: 0.9545 - val_loss: 0.1947 - val_accuracy: 0.9752 - val_precision: 0.9528 - val_recall: 1.0000
Epoch 4/19
122/122 [==============================] - 18s 151ms/step - loss: 0.1924 - accuracy: 0.9685 - precision: 0.9604 - recall: 0.9773 - val_loss: 0.1829 - val_accuracy: 0.9773 - val_precision: 0.9565 - val_recall: 1.0000
Epoch 5/19


In [43]:
load_saved_model = ('model/new_model2.h5')
live_model = tf.keras.models.load_model(str(load_saved_model), 
                                         custom_objects = {'KerasLayer':hub.KerasLayer})

In [46]:
final_loss, final_accuracy, precision, recall = live_model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))
print("Precision: {:.2f}%".format(precision * 100))
print("Recall: {:.2f}%".format(recall* 100))
print("F1-score: {:.2f}%".format((2*(precision*recall)/(precision+recall))* 100))

320/320 [==============================] - 50s 114ms/step - loss: 0.3841 - accuracy: 0.9392 - precision: 0.9634 - recall: 0.9650
Final loss: 0.38
Final accuracy: 93.92%
Precision: 96.34%
Recall: 96.50%
F1-score: 96.42%
